### Calcoliamo un indice di diversità per ciascun quartiere e ciascun anno del dataset
La discussione dell'indice di diversità è [sul wiki](https://github.com/spaghetti-open-data/-milano-melting-pot/wiki/Indici-di-diversit%C3%A0).

In [1]:
import json
dirPath = '/Users/albertocottica/Documents/GitHub/-milano-melting-pot/dati/'
with open (dirPath + 'map1Data_ultracompact.json', 'r') as jsonfile:
    data = json.load(jsonfile)
print(len(data))
print (data[:4])

1760
[{'Anno': '1999', 'IdNil': '1', 'eastas': 71, 'noraf': 102, 'ssaf': 141, 'seas': 309, 'ita': 17602, 'southas': 198, 'soam': 260, 'eu15': 663, 'westas': 124, 'noram': 147, 'eun13': 56, 'otheu': 134, 'ocean': 5, 'other': 5}, {'Anno': '1999', 'IdNil': '2', 'eastas': 105, 'noraf': 94, 'ssaf': 181, 'seas': 251, 'ita': 17523, 'southas': 147, 'soam': 243, 'eu15': 660, 'westas': 38, 'noram': 163, 'eun13': 41, 'otheu': 166, 'ocean': 8, 'other': 3}, {'Anno': '1999', 'IdNil': '3', 'eastas': 0, 'noraf': 0, 'ssaf': 2, 'seas': 6, 'ita': 32, 'southas': 3, 'soam': 1, 'eu15': 8, 'westas': 0, 'noram': 1, 'eun13': 0, 'otheu': 0, 'ocean': 0, 'other': 0}, {'Anno': '1999', 'IdNil': '4', 'eastas': 73, 'noraf': 103, 'ssaf': 141, 'seas': 298, 'ita': 14761, 'southas': 185, 'soam': 253, 'eu15': 421, 'westas': 108, 'noram': 85, 'eun13': 51, 'otheu': 172, 'ocean': 3, 'other': 2}]


Calcoliamo l'indice di segregazione. L'idea è questa: per ciascun anno e per ciascun quartiere devo trovare tutti i record con i numeri di tutte le nazionalità, e metterli in un vettore. A questo punto calcolo l'entropia del vettore, e salvo il risultato in un dizionario.

In [2]:
import math
def segregation_index(nats):
    '''
    (list) => float
    ritorna l'indice di segregazione associato a una popolazione divisa in componenti. 
    Ciascun componente ha una numerosità, che è un elemento di nats.
    '''
    base = len(nats) # the logarithm's base
    pop = sum(nats)
    index = 0
    for nat in nats:
        if nat == 0:
            nat = 0.000000001 # log(0) is undefined and breaks the code
        index += nat/pop * math.log(nat/pop, base)
    return -index

## test
mylist = [1,2,3,4,5,6,7,8,9]
print (segregation_index(mylist))

0.9329227160192092


Calcolo l'indice relativo alla città per la normalizzazione

In [7]:
Milano = []
for anno in range(1999,2019):
    transient = {} # la popolazione cittadina di ciascuna regione di provenienza va in un dizionario /
            # si sa mai che i valori non siano salvati in ordine
    newItem = {'Anno': anno}
    for item in data:
        if int(item['Anno']) == anno:
            for key in item:
                if key != 'Anno' and key != 'IdNil':
                    if key in transient:
                        transient[key] += int(item[key])
                    else:
                        transient[key] = int(item[key])
    thisYear = [] # dal dizionario, ora che ho le somme fatte con ordine, mi ricavo la lista...
    for key2 in transient:
        thisYear.append(transient[key2])
    newItem['H_res'] = segregation_index(thisYear) # ... e la passo alla funzione
    Milano.append(newItem) 
print (len(Milano))
print(Milano)
            

20
[{'Anno': 1999, 'H_res': 0.1707351253912768}, {'Anno': 2000, 'H_res': 0.18733096017229522}, {'Anno': 2001, 'H_res': 0.20592311820041492}, {'Anno': 2002, 'H_res': 0.21038275988824687}, {'Anno': 2003, 'H_res': 0.1804136925417503}, {'Anno': 2004, 'H_res': 0.22109279850688238}, {'Anno': 2005, 'H_res': 0.24355433748778152}, {'Anno': 2006, 'H_res': 0.2537292692927316}, {'Anno': 2007, 'H_res': 0.26129015987744303}, {'Anno': 2008, 'H_res': 0.26852125467179994}, {'Anno': 2009, 'H_res': 0.28711524486186696}, {'Anno': 2010, 'H_res': 0.30369110149773254}, {'Anno': 2011, 'H_res': 0.3204581012548873}, {'Anno': 2012, 'H_res': 0.341056538043024}, {'Anno': 2013, 'H_res': 0.34588060266491755}, {'Anno': 2014, 'H_res': 0.33427977185610763}, {'Anno': 2015, 'H_res': 0.3387367837063571}, {'Anno': 2016, 'H_res': 0.3390500396121212}, {'Anno': 2017, 'H_res': 0.34399668118754784}, {'Anno': 2018, 'H_res': 0.35054622343214326}]


L'indice di diversità a Milano è raddoppiato in 20 anni. Questo suggerisce di mantenere, per ciascun quartiere e per ciascun anno, sia il valore assoluto `H_res` che quello normalizzato `H_rn`. Il primo serve a tracciare l'evoluzione del quartiere negli anni; il secondo a posizionarlo rispetto alla città. 

Calcolo gli indici assoluti:

In [8]:
map3Data = []
for item in data:
    nationalities = []
    for key in item:
        if key != 'Anno' and key != 'IdNil':
            nationalities.append(int(item[key]))
    newitem = {'Anno': item['Anno'], 'IdNil': item['IdNil'], 'H_res': segregation_index(nationalities)}
    map3Data.append(newitem)
print(map3Data[:4])

[{'Anno': '1999', 'IdNil': '1', 'H_res': 0.2242727988555109}, {'Anno': '1999', 'IdNil': '2', 'H_res': 0.21539763514080623}, {'Anno': '1999', 'IdNil': '3', 'H_res': 0.48225983966002406}, {'Anno': '1999', 'IdNil': '4', 'H_res': 0.23053975335074206}]


Calcolo gli indici relativi.

In [10]:
for item in map3Data:
    anno = int(item['Anno'])
    for item2 in Milano:
        if item2['Anno'] == anno:
            item['H_rn'] = item['H_res'] - item2['H_res']
print(len(map3Data))
print(map3Data[:4])

1760
[{'Anno': '1999', 'IdNil': '1', 'H_res': 0.2242727988555109, 'H_rn': 0.0535376734642341}, {'Anno': '1999', 'IdNil': '2', 'H_res': 0.21539763514080623, 'H_rn': 0.04466250974952943}, {'Anno': '1999', 'IdNil': '3', 'H_res': 0.48225983966002406, 'H_rn': 0.3115247142687473}, {'Anno': '1999', 'IdNil': '4', 'H_res': 0.23053975335074206, 'H_rn': 0.059804627959465256}]


Salviamo...

In [11]:
with open (dirPath + 'map3Data.json', 'w') as jsonfile:
    json.dump(map3Data, jsonfile)
print('Done')

Done
